説明を読む感じでは、これはLangChain側で実装してもいいかなって感じはします
https://medium.com/@thomas_reid/llamaparse-rag-beats-all-comers-60948c6cc0e4

この記事を参考にしたコードを作成する
Rerankingと複数のpdfを読み込めるかをテストしながら実装してみる

In [55]:
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.cohere_rerank import CohereRerank

# setting up our LLM Settings
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")

In [8]:
import os
from pathlib import Path
from glob import glob

In [17]:
pdf_path = glob("./pdf/*.pdf")
len(pdf_path)

3

In [18]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    num_workers=len(pdf_path)
)

In [19]:
# parse PDF
documents = parser.load_data(pdf_path)

Parsing files:   0%|          | 0/3 [00:00<?, ?it/s]

Parsing files: 100%|██████████| 3/3 [02:07<00:00, 42.55s/it]


In [23]:
len(documents)

238

In [27]:
node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model="gpt-4o-mini"),
    num_worker=len(pdf_path)
)
nodes = node_parser.get_nodes_from_documents(documents)

0it [00:00, ?it/s]
2it [00:00, 1145.52it/s]
0it [00:00, ?it/s]
2it [00:00, 2631.31it/s]
1it [00:00, 1610.72it/s]
1it [00:00, 5322.72it/s]
1it [00:00, 2983.15it/s]
2it [00:00, 6775.94it/s]
1it [00:00, 2916.76it/s]
0it [00:00, ?it/s]
3it [00:00, 2997.36it/s]
18it [00:00, 45811.57it/s]
6it [00:00, 63710.95it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 3079.52it/s]
1it [00:00, 2755.78it/s]
4it [00:00, 27016.45it/s]
1it [00:00, 4999.17it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 874.00it/s]
2it [00:00, 12282.00it/s]
6it [00:00, 5524.88it/s]
7it [00:00, 23870.02it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 5683.34it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 7096.96it/s]
1it [00:00, 6452.78it/s]
1it [00:00, 15141.89it/s]
4it [00:00, 9264.06it/s]
3it [00:00, 4797.15it/s]
2it [00:00, 19239.93it/s]
4it [00:00, 28777.39it/s]


In [30]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
recursive_index = VectorStoreIndex(nodes=base_nodes + objects)

In [29]:
reranker = CohereRerank(top_n=5)

In [36]:
query_engine = recursive_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessor=[reranker],
    verbose=True
)

#### page_nodesを加える

In [ ]:
from copy import deepcopy
from llama_index.core.schema import TextNode

def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata)
            )
            nodes.append(node)
    return nodes

page_nodes = get_page_nodes(documents)

recursive_index_page = VectorStoreIndex(nodes=base_nodes + objects + page_nodes)
recursive_pages_query_engine = recursive_index_page.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
    verbose=True
)


In [63]:
import wandb
import weave

wandb.init(project="llama-parse-test")

wandb: Currently logged in as: y-hiroki-rad to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [65]:
@weave.op()
def get_pages_response(query):
    response = recursive_pages_query_engine.query(query)
    print(response)

### テスト

In [66]:
@weave.op()
def get_response(query):
    response = query_engine.query(query)
    print(response)

In [ ]:
"""
質問1: 4℃グループの人材育成プログラムの根幹は何ですか？
回答: サステナブル経営の根幹である『人財』に対するグループの姿勢を表すものです
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==


4℃グループの人材育成プログラムの根幹は、持続的に人を育てるための教育を体系化し、世代を超えて実施することにあります。このプログラムは、教育を期待と愛情を伝える手段として捉え、企業の持続的成長を支えるために人財を大切にする組織風土を育むことに注力しています。
== used page_nodes ==


In [56]:
"""
質問2: 4℃グループが事業継続のために策定している計画にはどのようなものがありますか？
回答: 台風や大雨による店舗の営業停止、物流プロセスの寸断などのリスクに対応するため、事業継続計画を策定しています
"""
query = "4℃グループが事業継続のために策定している計画にはどのようなものがありますか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
4℃グループは、事業継続のために第7次中期経営計画を策定しています。この計画では、環境変化への対応を前提に顧客提供価値を追求し、将来の成長基盤を構築することを目指しています。具体的には、「4℃」ブランドの価値を具現化し、高収益ビジネスモデルの再構築を行うことが中心となっています。また、ファッションジュエリーの再構築やECチャネルの拡充、OMO戦略の推進なども含まれています。さらに、アパレル事業の出店拡大やM&A、海外展開、新規事業の実行を通じて成長を図る計画もあります。
== used page_nodes ==
4℃グループは、事業継続のために第7次中期経営計画を策定しています。この計画では、「環境変化への対応」を前提に「顧客提供価値」を追求し、将来の飛躍に向けた成長基盤の構築を目指しています。具体的には、ブランド事業の価値を具現化し、高収益ビジネスモデルの再構築を行うことや、ファッションジュエリーの再構築、ECチャネルの拡充、OMO戦略の推進などが含まれています。また、アパレル事業においては、店舗の出店拡大や安定的な配当、顧客化の深耕なども計画されています。これらの取り組みを通じて、持続的な成長を実現することを目指しています。


In [57]:
"""
質問3: 4℃グループが人的資本経営の実践のために推進していることは何ですか？
回答: 従業員のwell-beingを重視し、健康経営、ワークライフバランス、多様性尊重などを推進しています
"""
query = "4℃グループが人的資本経営の実践のために推進していることは何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
Retrieval entering 97965058-fa11-43a3-8f4a-9553ae1fbc1d: TextNode
Retrieving from object TextNode with query 4℃グループが人的資本経営の実践のために推進していることは何ですか？
4℃グループは人的資本経営の実践のために、次のような取り組みを推進しています。まず、人間尊重を基本理念とし、多様な人財の育成に積極的に取り組むことで企業価値を高め、持続可能な社会の実現を目指しています。次世代リーダーの育成を目的とした教育プログラムや、女性活躍支援プログラムを実施し、特に2030年度には女性管理職比率40.0％を目指しています。

また、従業員の能力開発とキャリア形成を促進するために、教育体系を整備し、テレワーク制度の導入やワークライフバランスの向上にも取り組んでいます。さらに、心理的安全性を確保し、人権を尊重する企業文化の醸成を図ることも重要な課題としています。これらの取り組みを通じて、従業員がいきいきと働ける環境を整え、持続的な成長を支援しています。
== used page_nodes ==
Retrieval entering 97965058-fa11-43a3-8f4a-9553ae1fbc1d: TextNode
Retrieving from object TextNode with query 4℃グループが人的資本経営の実践のために推進していることは何ですか？
4℃グループは、人的資本経営の実践のために、従業員の健康を重視し、テレワーク制度の導入や残業時間の削減を通じてワークライフバランスの向上に取り組んでいます。また、身体・精神・社会的に満たされた状態で働ける環境を目指し、自己実現と成長の支援、働きやすい職場づくり、多様性や個性の尊重を推進しています。さらに、次世代リーダーの育成や女性活躍支援プログラムなど、教育・研修を通じて従業員の能力開発とキャリア形成を促進しています。


In [58]:
"""
質問3: IHIグループが事業開発と技術開発で重視していることは何ですか？
回答: バリューチェーン視点での事業開発と、将来のコアとなる可能性の高い技術の獲得を重視しています
"""
query = "IHIグループが事業開発と技術開発で重視していることは何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
IHIグループが事業開発と技術開発で重視しているのは、バリューチェーン視点を持った技術開発と、地域戦略に基づく新たなビジネスモデルの構築です。また、競争優位性を確保するために、社内外の技術動向を把握し、必要な技術を迅速に獲得することも重要視されています。さらに、環境変化に対応した独創的な研究開発を行い、将来のビジョンやシナリオを描くことが求められています。
== used page_nodes ==
IHIグループが事業開発と技術開発で重視しているのは、バリューチェーン視点を持った技術開発と、地域戦略を組み合わせたスピード感とスケール感のある事業開発です。また、社内外の技術動向を把握し、必要な技術を迅速に獲得することも重要なミッションとされています。これにより、競争優位性を確保し、持続可能な社会の実現に向けた新たなビジネスモデルを構築することを目指しています。


In [59]:
"""
質問1: 日産が目指すサステナビリティとは何ですか？
回答: よりクリーンで、安全で、インクルーシブな、誰もが共生できる世界の実現を推進し、真にサステナブルな企業となることを目指しています
"""
query = "日産が目指すサステナビリティとは何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
日産が目指すサステナビリティは、「人とクルマと自然の共生」を理念とし、エネルギーや資源の使用効率を高め、多様性や循環を促進しながら、革新的な商品やサービスを提供することです。具体的には、事業活動やクルマのライフサイクル全体での環境への依存と負荷を自然が吸収可能なレベルに抑え、豊かな自然資産を次世代に引き継ぐことを究極のゴールとしています。また、安全で持続可能なモビリティを提供し、気候変動や資源問題への対応を重視し、社会に価値を提供することが重要とされています。
== used page_nodes ==
日産が目指すサステナビリティは、コーポレートパーパス「人々の生活を豊かに。イノベーションをドライブし続ける」に基づき、信頼される企業として革新的なクルマやサービスを提供し、すべてのステークホルダーに優れた価値を提供することです。具体的には、よりクリーンで安全、インクルーシブな世界の実現を推進し、気候変動や資源依存、大気品質・水などの環境課題に対してポジティブな影響を与えることを目指しています。また、エネルギーや資源の使用効率を高め、多様性や循環を促進しながら、持続可能なモビリティ社会の発展に貢献することが重要視されています。最終的には、事業活動やクルマのライフサイクル全体での環境への依存と負荷を自然が吸収可能なレベルに抑えることを究極のゴールとしています。


In [60]:
"""
質問3: 日産が従業員の能力開発のために行っていることは何ですか？
回答: グローバル共通のトレーニングプラットフォームの導入、重要なスキル習得の機会提供、テクノロジーを活用した学習体験の充実化、実効性のある学びの機会創出、タレントアクセラレーションプログラムの見直しと再構築などを通じて、従業員の能力開発を推進しています
"""
query = "日産が従業員の能力開発のために行っていることは何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
日産は従業員の能力開発のために、さまざまなプログラムを提供しています。具体的には、マネジメントスキル、ビジネススキル、リーダーシップ開発などの多様な研修を実施し、オンラインやオンデマンドでの講座も拡充しています。また、キャリア形成を支援するために、上司とのキャリア面談を年3回行い、従業員の自発的な学びを促進しています。さらに、コンピテンシー評価やパフォーマンス評価を導入し、専門性の強化を目的としたエキスパートリーダー制度も設けています。リモートワーク環境下でも学びの機会を提供するために、2万種類以上のe-ラーニングコンテンツを用意し、デジタル学習の基盤を強化しています。
== used page_nodes ==
日産は従業員の能力開発のために、以下の取り組みを行っています。

1. **能力開発プログラム**: 従業員のエンプロイアビリティを向上させるためのプログラムを実施し、社会の変革に対応できる人材を育成しています。

2. **自発的学習の促進**: 従業員が自発的に学ぶ姿勢を大切にし、企業文化の醸成と人材育成に努めています。

3. **キャリア面談**: 全従業員を対象に年3回、上司とのキャリア面談を実施し、キャリア形成をサポートしています。

4. **人事制度の改善**: コンピテンシー評価やパフォーマンス評価を導入し、従業員のスキルや知識に基づく評価を行っています。

5. **e-ラーニングの提供**: 2万種類以上のe-ラーニングコンテンツをグローバルに提供し、リモートワーク環境下でも効率的な学びの機会を提供しています。

6. **日産ラーニングセンター**: 技術革新に対応するため、リーダー層の育成や次世代への技術・技能伝承を目的とした学習機会を提供しています。

7. **シニアパートナー制度**: 高い専門性と経験を有するシニア従業員が継続的に活躍できるよう、柔軟な働き方を整備しています。

8. **専門性の強化**: エキスパートリーダー制度を導入し、さまざまな領域での専門性を強化しています。

これらの取り組みにより、従業員のスキル向上とキャリア形成を支援しています。
